In [1]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
from datetime import datetime, timedelta
from utils import format_number

from config import LAST_N_DAYS, COL_NAMES_TO_INCLUDE

pd.options.display.float_format = "{:,.2f}".format

In [2]:
def extract_source(source_string):
    source_list = source_string.split("-")
    if len(source_list) > 1:
        return source_list[1].strip()  # strip() is used to remove any leading/trailing spaces
    else:
        return source_string.strip()
    
def cleanup_string(source_string):
    return source_string.replace(" ", "").lower()

def merge_dfs(df_to_merge, key="app_name_join", cols=COL_NAMES_TO_INCLUDE):
    df_combined = pd.merge(df_choice, df_to_merge, on=key).merge(df_tvl, on=key).merge(df_op_distribution, on=key)
    return df_combined[cols]

def calculate_metrics(df):
    inc_cols = df.filter(like='incremental_').columns
    df = df.assign(**{f'incremental_{col.split("_")[1]}_annualized_per_op': df[col] * 365 / df["net_op_deployed"] for col in inc_cols})
    df["net_tvl_per_op"] = df["cumul_last_price_net_dollar_flow"] / df["net_op_deployed"]

    return df

# Incentive Program Summary
Status of programs live, completed and to be announced by season.

In [3]:
df_info = pd.read_csv("inputs/" + "op_incentive_program_info" + ".csv")

In [4]:
# convert to datetime
df_info["start_date"] = pd.to_datetime( df_info["Announced On"].fillna(df_info["Start Date"]) )
df_info["end_date"] = pd.to_datetime( df_info["End Date"] )

# convert program status into ordered categorical type
cat_size_order = CategoricalDtype(
    ["Live ‎🔥", "Coming soon ‎⏳", "Completed"], 
    ordered=True
)
df_info["Status"] = df_info["Status"].astype(cat_size_order)

for i in ['GovFund','GovFund Growth Experiments','All Programs']:
    # Assign the filters
    if i == 'GovFund':
        filter_name = " - GovFund Only"
        df_choice = df_info[df_info['Source'] != 'Partner Fund'].copy()
    elif i == 'GovFund Growth Experiments':
        filter_name = " - GovFund Growth Exp."
        df_choice = df_info[df_info['Source'] != 'Partner Fund'].copy()
        df_choice =  df_choice[df_choice['Incentive / Growth Program Included?'] == 'Yes']
    else:
        filter_name = ""
        df_choice = df_info.copy()

    # clean up for columns needed
    df_choice = df_choice[["Source","Status","# OP Allocated","App Name","start_date", "end_date", "App Name Map Override"]]
    summary = pd.pivot_table(df_choice, values=["# OP Allocated", "App Name"], index=["Status", "Source"], \
        aggfunc={"# OP Allocated":"sum", "App Name":"count"})

    subtotal_name = "Subtotal" + filter_name
    # calculate subtotals on program status
    result=pd.concat([summary,summary.groupby(level=0).sum().assign(item_name=subtotal_name).set_index("item_name",append=True)]).sort_index(level=[0,1])
    result = result.sort_index(level=[0, 1], ascending=[True, False])

    # add grand total to summary
    result.loc[("Grand Total"), "# OP Allocated"] = summary["# OP Allocated"].sum()
    result.loc[("Grand Total"), "App Name"] = summary["App Name"].sum()

    # cleanup display
    result["# Programs"] = result["App Name"].astype(int)
    result["# OP Allocated (M)"] = result["# OP Allocated"].apply(format_number)

    # calculate percentage of total
    result.loc[(slice(None), subtotal_name), "# OP Allocated"] / summary["# OP Allocated"].sum()
    result["% OP Allocated"] = round(result.loc[(slice(None), subtotal_name), "# OP Allocated"] / summary["# OP Allocated"].sum() * 100).\
        astype(str).replace("\.0", "", regex=True) + "%"
    result["% OP Allocated"].fillna("-", inplace=True)

    result = result.replace((0, "0.0M"), "-")
    print(i)
    display(result.drop(columns=["# OP Allocated", "App Name"]))
    print()

GovFund


# Programs # OP Allocated (M)  \
Status         Source                                                  
Live ‎🔥        Subtotal - GovFund Only         35              41.2M   
               Governance - Season 3            -                0.0   
               Governance - Season 2           12               7.2M   
               Governance - Season 1            9               4.5M   
               Governance - Phase 0            14              29.5M   
Coming soon ‎⏳ Subtotal - GovFund Only         54              14.4M   
               Governance - Season 3           22               2.5M   
               Governance - Season 2           18               5.7M   
               Governance - Season 1            7               2.1M   
               Governance - Phase 0             7               4.1M   
Completed      Subtotal - GovFund Only          5               2.6M   
               Governance - Season 3            -                0.0   
               Governance - Season 2            1             240.0K   
               Governance - Season 1            -                0.0   
               Governance - Phase 0             4               2.4M   
Grand Total                                    94              58.2M   

                                       % OP Allocated  
Status         Source                                  
Live ‎🔥        Subtotal - GovFund Only            71%  
               Governance - Season 3                -  
               Governance - Season 2                -  
               Governance - Season 1                -  
               Governance - Phase 0                 -  
Coming soon ‎⏳ Subtotal - GovFund Only            25%  
               Governance - Season 3                -  
               Governance - Season 2                -  
               Governance - Season 1                -  
               Governance - Phase 0                 -  
Completed      Subtotal - GovFund Only             5%  
               Governance - Season 3                -  
               Governance - Season 2                -  
               Governance - Season 1                -  
               Governance - Phase 0                 -  
Grand Total                                         -


GovFund Growth Experiments


# Programs # OP Allocated (M)  \
Status         Source                                                         
Live ‎🔥        Subtotal - GovFund Growth Exp.         33              41.1M   
               Governance - Season 3                   -                0.0   
               Governance - Season 2                  10               7.1M   
               Governance - Season 1                   9               4.5M   
               Governance - Phase 0                   14              29.5M   
Coming soon ‎⏳ Subtotal - GovFund Growth Exp.         32              10.9M   
               Governance - Season 3                  12               2.1M   
               Governance - Season 2                  13               5.2M   
               Governance - Season 1                   4               1.3M   
               Governance - Phase 0                    3               2.2M   
Completed      Subtotal - GovFund Growth Exp.          5               2.6M   
               Governance - Season 3                   -                0.0   
               Governance - Season 2                   1             240.0K   
               Governance - Season 1                   -                0.0   
               Governance - Phase 0                    4               2.4M   
Grand Total                                           70              54.6M   

                                              % OP Allocated  
Status         Source                                         
Live ‎🔥        Subtotal - GovFund Growth Exp.            75%  
               Governance - Season 3                       -  
               Governance - Season 2                       -  
               Governance - Season 1                       -  
               Governance - Phase 0                        -  
Coming soon ‎⏳ Subtotal - GovFund Growth Exp.            20%  
               Governance - Season 3                       -  
               Governance - Season 2                       -  
               Governance - Season 1                       -  
               Governance - Phase 0                        -  
Completed      Subtotal - GovFund Growth Exp.             5%  
               Governance - Season 3                       -  
               Governance - Season 2                       -  
               Governance - Season 1                       -  
               Governance - Phase 0                        -  
Grand Total                                                -


All Programs


# Programs # OP Allocated (M)  \
Status         Source                                                
Live ‎🔥        Subtotal                      35              41.2M   
               Partner Fund                   -                0.0   
               Governance - Season 3          -                0.0   
               Governance - Season 2         12               7.2M   
               Governance - Season 1          9               4.5M   
               Governance - Phase 0          14              29.5M   
Coming soon ‎⏳ Subtotal                      54              14.4M   
               Partner Fund                   -                0.0   
               Governance - Season 3         22               2.5M   
               Governance - Season 2         18               5.7M   
               Governance - Season 1          7               2.1M   
               Governance - Phase 0           7               4.1M   
Completed      Subtotal                       8              11.1M   
               Partner Fund                   3               8.4M   
               Governance - Season 3          -                0.0   
               Governance - Season 2          1             240.0K   
               Governance - Season 1          -                0.0   
               Governance - Phase 0           4               2.4M   
Grand Total                                  97              66.7M   

                                     % OP Allocated  
Status         Source                                
Live ‎🔥        Subtotal                         62%  
               Partner Fund                       -  
               Governance - Season 3              -  
               Governance - Season 2              -  
               Governance - Season 1              -  
               Governance - Phase 0               -  
Coming soon ‎⏳ Subtotal                         22%  
               Partner Fund                       -  
               Governance - Season 3              -  
               Governance - Season 2              -  
               Governance - Season 1              -  
               Governance - Phase 0               -  
Completed      Subtotal                         17%  
               Partner Fund                       -  
               Governance - Season 3              -  
               Governance - Season 2              -  
               Governance - Season 1              -  
               Governance - Phase 0               -  
Grand Total                                       -

In [5]:
# display new programs in last 30 days
df_new_programs = df_choice[df_choice["start_date"]>pd.Timestamp("today") - timedelta(days = LAST_N_DAYS)].sort_values(by="start_date", ascending=False)
if not df_new_programs.empty:
    df_new_programs["end_date"].fillna("-", inplace=True)
    display(df_new_programs.drop("App Name Map Override", axis=1))

,Source,Status,# OP Allocated,App Name,start_date,end_date
67,NaN,Live ‎🔥,NaN,Quests on Coinbase Wallet,2023-03-09,-
81,Governance - Season 2,Live ‎🔥,"504,000.00",Sushiswap,2023-03-03,-
11,Governance - Season 2,Live ‎🔥,"33,000.00",Bankless Academy,2023-02-22,-
8,Governance - Season 2,Live ‎🔥,"250,000.00",Angle,2023-02-20,-
93,Governance - Season 1,Live ‎🔥,"300,000.00",WardenSwap,2023-02-17,-
68,Governance - Season 2,Live ‎🔥,"420,069.00",Rainbow Wallet,2023-02-13,-


In [6]:
# display completed programs in last 30 days
df_completed = df_choice[(df_choice['Status'] == 'Completed') & (df_choice['end_date']>pd.Timestamp('today') - timedelta(days = LAST_N_DAYS))]\
        .sort_values(by='start_date', ascending=False)
if not df_completed.empty:
        display(df_completed.drop("App Name Map Override", axis=1))

,Source,Status,# OP Allocated,App Name,start_date,end_date
69,Governance - Season 2,Completed,"240,000.00",Revert Finance,2022-11-03,2023-03-06


# Usage and TVL Attribution
To combine all sources of data together

### `DF_INFO` Cleanup

In [7]:
# create app_name_join, coalesce with app name override map, app name and remove any space
df_choice["app_name_join"] = df_choice["App Name Map Override"].fillna(df_choice["App Name"])
df_choice["app_name_join"] = df_choice["app_name_join"].apply(cleanup_string)

# apply the function to the "Source" column and create a new column "Source Extracted"
df_choice["source_cleanup"] = df_choice["Source"].astype(str).apply(extract_source)
df_choice["source_cleanup"] = df_choice["source_cleanup"].apply(cleanup_string)

df_info_summary = df_choice.groupby(["app_name_join", "App Name"]).sum()["# OP Allocated"].reset_index()

### `DF_USAGE` Cleanup

In [8]:
df_usage = pd.read_csv("csv_outputs/" + "dune_op_program_performance_summary" + ".csv")
df_usage["app_name_join"] = df_usage["app_name"].apply(cleanup_string)

df_usage["op_source_length"] = df_usage["op_source"].str.split(',').apply(len)

In [9]:
df_usage_single = df_usage[df_usage["op_source_length"] == 1]
df_usage_multiple = df_usage[df_usage["op_source_length"] > 1]

### `DF_TVL` Cleanup

In [10]:
df_tvl = pd.read_csv('csv_outputs/op_summer_latest_stats.csv')
df_tvl = df_tvl[df_tvl["include_in_summary"] == 1]

# cleanup
df_tvl["app_name_join"] = df_tvl["parent_protocol"].apply(cleanup_string)
df_tvl = df_tvl.groupby("app_name_join").sum()["cumul_last_price_net_dollar_flow"].reset_index()

### `DF_OP_DISTRIBUTION` Cleanup

In [11]:
df_op_distribution = pd.read_csv("csv_outputs/dune_op_distribution_type.csv")
df_op_distribution["net_op_deployed"] = (df_op_distribution["op_deployed"] - df_op_distribution["op_from_other_projects"]).astype(float)
df_op_distribution["app_name_join"] = df_op_distribution["from_name"].apply(cleanup_string)

df_op_distribution = df_op_distribution.groupby("app_name_join").sum()[["op_deployed", "net_op_deployed"]].reset_index()

### Combine Data

In [12]:
agg_dict = {
    "# OP Allocated": "sum",
    "net_op_deployed": "sum",
    "incremental_addr_per_day": "sum",
    "incremental_txs_per_day": "sum",
    "incremental_gas_fee_eth_per_day": "sum",
    "cumul_last_price_net_dollar_flow": "sum"
}

#### By Fund Source

In [47]:
# single programs
df_combined = merge_dfs(df_usage_single)
result_fund_source = df_combined.groupby("Source").agg(agg_dict)
result_fund_source.sort_values("Source").reset_index()

# calculate metrics
result_fund_source = calculate_metrics(result_fund_source)
result_fund_source = result_fund_source.reset_index()
result_fund_source[~result_fund_source["Source"].isin(["Governance - Season 3", "Partner Fund"])]


,Source,# OP Allocated,net_op_deployed,incremental_addr_per_day,incremental_txs_per_day,incremental_gas_fee_eth_per_day,cumul_last_price_net_dollar_flow,incremental_addr_annualized_per_op,incremental_txs_annualized_per_op,incremental_gas_annualized_per_op,net_tvl_per_op
0,Governance - Phase 0,"28,800,000.00","20,825,762.00","8,621.51","9,719.46",67.94,"203,509,034.14",0.15,0.17,0.00,9.77
1,Governance - Season 1,"3,300,000.00","1,763,165.00","1,235.87","3,353.81",10.18,"110,691,311.52",0.26,0.69,0.00,62.78
2,Governance - Season 2,"1,490,000.00","906,065.00",260.38,685.12,2.43,"17,780,032.12",0.10,0.28,0.00,19.62


In [14]:
# multiple programs
df_tvl_multiple = df_tvl.groupby("app_name_join").sum()["cumul_last_price_net_dollar_flow"]
df_choice_multiple = df_choice.groupby("app_name_join").sum()["# OP Allocated"]
df_op_distribution_multiple = df_op_distribution.groupby("app_name_join").sum()["net_op_deployed"]
df_combined_multiple = pd.merge(df_usage_multiple, df_choice_multiple, on="app_name_join").merge(df_op_distribution_multiple, on="app_name_join").merge(df_tvl_multiple, on="app_name_join")
df_combined_multiple

# calculate metrics
result_fund_source_multiple = calculate_metrics(df_combined_multiple)
cols = [
    "op_source",
    "app_name",
    "# OP Allocated",
    "net_op_deployed",
    "incremental_addr_per_day",
    "incremental_txs_per_day",
    "incremental_gas_fee_eth_per_day",
    "cumul_last_price_net_dollar_flow"
]
result_fund_source_multiple = result_fund_source_multiple[cols]

# calculate metrics
result_fund_source_multiple = calculate_metrics(result_fund_source_multiple)
result_fund_source_multiple


,op_source,app_name,# OP Allocated,net_op_deployed,incremental_addr_per_day,incremental_txs_per_day,incremental_gas_fee_eth_per_day,cumul_last_price_net_dollar_flow,incremental_addr_annualized_per_op,incremental_txs_annualized_per_op,incremental_gas_annualized_per_op,net_tvl_per_op
0,"['Partner Fund', 'Season 2']",Velodrome,"7,000,000.00","5,054,767.00","1,718.54","8,045.35",19.45,"201,955,598.20",0.12,0.58,0.00,39.95
1,"['Phase 0', 'Season 2']",Thales,"1,450,000.00","1,048,991.00",209.03,745.57,0.64,"-122,771.74",0.07,0.26,0.00,-0.12
2,"['Season 2', 'Partner Fund']",PoolTogether,"1,000,000.00","842,488.00","1,688.25","1,910.07",0.14,"25,209,238.41",0.73,0.83,0.00,29.92


In [32]:
# by app 
df_info["app_name_join"] = df_info["App Name Map Override"].fillna(df_info["App Name"])
df_info["app_name_join"] = df_info["app_name_join"].apply(cleanup_string)

df_choice_combined = df_info.groupby("app_name_join").sum()
df_combined = pd.merge(df_choice_combined, df_usage, on="app_name_join").merge(df_tvl, on="app_name_join").merge(df_op_distribution, on="app_name_join")
df_combined = df_combined[cols]

result_app = df_combined.groupby("app_name").agg(agg_dict)

# calculate metrics
result_app = calculate_metrics(result_app)


In [38]:
result_app = result_app.reset_index()

In [41]:
# sort by tvl 
cols = [
    "app_name",
    "# OP Allocated",
    "net_op_deployed",
    "cumul_last_price_net_dollar_flow",
    "net_tvl_per_op"
]
result_app[cols].sort_values("cumul_last_price_net_dollar_flow", ascending=False).reset_index(drop=True).head(10)

,app_name,# OP Allocated,net_op_deployed,cumul_last_price_net_dollar_flow,net_tvl_per_op
0,Velodrome,"7,000,000.00","5,054,767.00","201,955,598.20",39.95
1,Synthetix,"9,000,000.00","4,919,425.00","83,886,839.97",17.05
2,Aave,"5,300,000.00","4,820,781.00","77,329,625.73",16.04
3,Rocket Pool,"600,000.00","205,002.00","36,404,537.59",177.58
4,Beefy Finance,"650,000.00","281,962.00","34,600,187.71",122.71
5,Stargate Finance,"1,000,000.00","469,725.00","26,951,682.20",57.38
6,Beethoven X,"500,000.00","164,703.00","26,250,099.00",159.38
7,PoolTogether,"1,000,000.00","842,488.00","25,209,238.41",29.92
8,Pika Protocol,"900,000.00","672,632.00","10,999,054.19",16.35
9,dHedge,"350,000.00","202,257.00","8,728,060.05",43.15


In [43]:
# sort by txs
cols = [
    "app_name",
    "# OP Allocated",
    "net_op_deployed",
    "incremental_txs_per_day",
    "incremental_txs_annualized_per_op"
]
result_app[cols].sort_values("incremental_txs_per_day", ascending=False).reset_index(drop=True).head(10)

,app_name,# OP Allocated,net_op_deployed,incremental_txs_per_day,incremental_txs_annualized_per_op
0,Velodrome,"7,000,000.00","5,054,767.00","8,045.35",0.58
1,Pika Protocol,"900,000.00","672,632.00","4,781.95",2.59
2,Rubicon,"1,100,000.00","791,082.00","4,110.26",1.90
3,Synthetix,"9,000,000.00","4,919,425.00","4,092.16",0.30
4,Aave,"5,300,000.00","4,820,781.00","3,122.79",0.24
5,Hop Protocol,"1,000,000.00","152,602.00","3,003.30",7.18
6,Beethoven X,"500,000.00","164,703.00","2,297.19",5.09
7,PoolTogether,"1,000,000.00","842,488.00","1,910.07",0.83
8,Stargate Finance,"1,000,000.00","469,725.00","1,306.27",1.02
9,QiDao,"750,000.00","342,865.00",930.99,0.99
